In [1]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
import numpy as np
from mufc_fun import run_federated_clustering
from sklearn.cluster import KMeans
import math
from model import MyKmeans

def custom_generate_data(data_input, num_clusters, num_clients, split='iid', k_prime=None):
    """
    Process and split the data into a format expected by the federated clustering method.
    """
    json_data = {}
    json_data['full_data'] = data_input

    # Perform K-means on the full dataset to find the initial clusters
    clf = KMeans(n_clusters=num_clusters).fit(data_input)
    kmeans_loss = clf.inertia_
    kmeans_label = clf.labels_
    json_data['kmeans_loss'] = kmeans_loss

    # Initialize the data structure for each client
    for i in range(num_clients):
        json_data['client_' + str(i)] = []

    # Split the data into clients based on the selected method
    if split == 'iid':
        for k in range(num_clusters):
            data_cluster = data_input[kmeans_label == k, :]
            size_per_client = math.floor(data_cluster.shape[0] / num_clients)
            for i in range(num_clients - 1):
                json_data['client_' + str(i)].append(data_cluster[i * size_per_client:(i + 1) * size_per_client, :])
            # The last client gets the remaining data
            json_data['client_' + str(num_clients - 1)].append(data_cluster[(num_clients - 1) * size_per_client:, :])

        # Concatenate all data pieces for each client
        for i in range(num_clients):
            if len(json_data['client_' + str(i)]) > 0:
                json_data['client_' + str(i)] = np.concatenate(json_data['client_' + str(i)], axis=0)

    else:
        raise NotImplementedError("Only 'iid' split is currently implemented in this custom function.")

    return json_data

# Load the Iris dataset
iris = load_iris()
data = iris.data

# Normalize the data
scaler = StandardScaler()
data = scaler.fit_transform(data)

# Prepare the dataset dictionary
dataset = {
    'full_data': data,
    'num_clusters': 3,  # There are 3 clusters in the Iris dataset
}

# Use the custom function to split the data among clients (reduce clients to 5 or 10)
dataset.update(custom_generate_data(data, num_clusters=3, num_clients=10, split='iid', k_prime=3))
results = run_federated_clustering(dataset, num_clusters=3, num_clients=10, split='iid', k_prime=3, max_iters = 10)
print("Final Cluster Centroids:", results["final_clusters"])


Use fixed eps
eps = 0.08164965809277261, n = 150
============================== Trial 0 ==============================
===== MUFC training =====


100%|██████████| 10/10 [00:00<00:00, 361.64it/s]

===== Loss after training =====
mufc optimal loss: 197.3799055330865; global induced: 141.77501554466804, ratio: 0.7182849498370163; local induced: 157.43430980998994, ratio: 0.7976207577199366
mufc time used: 0.07852673530578613 s
Final Cluster Centroids: [[ 1.06858053  0.00602436  1.00166953  1.04926851]
 [-1.01965504  0.85325568 -1.30793281 -1.25894823]
 [ 0.04078993 -0.75216424  0.36109594  0.2950691 ]]
